In [ ]:
# Global params 
from datetime import datetime

def nu_t():
    return datetime.utcnow().strftime("%H:%M:%S:%f")

my_detector = 'DS-20K'

In [ ]:
# %%timeit
from SNEWS_PT.snews_pub import CoincidenceTier, Publisher

with Publisher() as pub:
    message = CoincidenceTier(detector_name=my_detector, nu_time=nu_t(), p_val = 0.98,).message()
    pub.send(message)

In [ ]:
%%time
from SNEWS_PT.snews_pub import SignificanceTier, Publisher

with Publisher() as pub:
    message = SignificanceTier(detector_name=my_detector, p_values=[0.78,0.67,0.75,0.56],nu_time = nu_t()).message()
    pub.send(message)


In [ ]:
from SNEWS_PT.snews_pub import TimingTier, Publisher

with Publisher() as pub:
    message = TimingTier(detector_name=my_detector, nu_time=nu_t(), timing_series = ['16:23:24"4546','16:23:24"4666','16:23:24"4756']).message()
    pub.send(message)

In [ ]:
from SNEWS_PT.snews_pub import Retraction, Publisher

with Publisher() as pub:
    message = Retraction(detector_name = my_detector, which_tier='CoincidenceTier', n_retract_latest=5, retraction_reason='someone spilled coffee on the DAQ').message()
    pub.send(message)